In [1]:
#! pip install pyensae
! pip install jyquickhelper

In [2]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Objectifs des prochaines séances

Connaissez-vous l'application [Pocket](https://getpocket.com/) ? C'est une application qui simplifie le bookmarking. Elle prend la forme d'une extension Chrome / Firefox. Quand on tombe sur un site intéressant, on peut le bookmaker, et ajouter, ou non, des tags pour "qualifier" le contenu.

Cette application répond au besoin de conserver le contenu web pertinent et de le classer.

Au cours des prochaines séances, nous allons construire un outil de machine learning qui :
- se connecte à un compte pocket
- récupère les sites bookmarqués et les tags éventuels
- à partir des articles taggés, prédit les meilleurs tags des sites non-taggés
- tag les articles non taggés

Bref, nous allons concevoir un programme de classification automatique des articles !

## Objectif de la séance

- Créer un compte Pocket
- S'authentifier auprès de l'API
- Populer le compte avec des données via l'API
- Récupérer les données via l'API
- Scraper les sites bookmarqués pour enrichir les données

## Mais c'est quoi une API ?

On vous explique tout ici :  
- [Définition](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#definition)
- [Les API qui existent](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#les-api-qui-existent)
- [Comment parler à une API ?](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#comment-parler-a-une-api)

Donc un API est une interface permettant de _communiquer_ avec une application. En général, on veut récupérer des données. Donc la communication consiste à envoyer une requete HTTP (le plus souvent GET ou POST) et à récupérer des données, souvent au format json. Ici nous voulons récupérer des données d'un compte utilisateur pocket. 

Pour savoir comment on communique précisément avec l'API de pocket, il n'y a pas de secret : il faut lire la doc de ceux qui l'ont codée. On vous a simplifié un peu les étapes ci-après.

## Création d'un compte Pocket

Créer un compte sur https://getpocket.com/signup?ep=4. Il n'y a pas de vérification d'email, donc vous pouvez mettre un faux mail.

Aller sur la _console developer_ de pocket: https://getpocket.com/developer/apps/index.php

<img src="./console_developer_pocket.png" width="450"/>

Cliquer sur CREATE AN APPLICATION

Compléter le formulaire comme suit (vous pouvez changer le nom de l'application et la description)

<img src="./screen_consumer_key.png" width="450"/>

Cliquer sur CREATE APPLICATION

In [67]:
CONSUMER_KEY = input("Insérer ici le CONSUMER KEY de la plateform WEB ")

Insérer ici le CONSUMER KEY de la plateform WEB 71295-1dd253e18e7de215267b4bd4


<img src="./screen_consumer_key2.png" width="450"/>

Vous en aurez besoin pour vous connecter à l'API de pocket.

## Authentification

D'abord il faut s'[authentifier](https://getpocket.com/developer/docs/authentication)

Protocole utilisé ici : [OAUTH2](https://tools.ietf.org/html/rfc6749) (très classique). 

<img src="./screen_oauth2.png"/>

6 étapes donc avant d'avoir le droit de récupérer les données. De temps en temps, il existe une librairie python. C'est notre cas : https://github.com/tapanpandita/pocket. On va s'en servir pour s'authentifier. Mais pas pour récupérer les données (elle n'est plus à jour pour faire ça).

In [68]:
! pip install pocket

### Etape 1 : Obtenir un code d'authorisation => get_request_token

In [69]:
import pocket
from pocket import Pocket

REDIRECT_URI = "http://localhost:8888/redirect"
# c'est l'url à laquelle vous allez rediriger l'utilisateur (ici, vous) après que pocket a authentifié l'utilisateur (vous)
REQUEST_TOKEN = Pocket.get_request_token(consumer_key=CONSUMER_KEY, redirect_uri=REDIRECT_URI)
print(REQUEST_TOKEN)

72db9ea4-cbb3-a69d-d51b-3c7202


### Etape 2: Authoriser l'accès

In [70]:
url = "https://getpocket.com/auth/authorize?request_token={0}&redirect_uri={1}".format(REQUEST_TOKEN, REDIRECT_URI)
print("Aller à l'url : \n" + url)

Aller à l'url : 
https://getpocket.com/auth/authorize?request_token=72db9ea4-cbb3-a69d-d51b-3c7202&redirect_uri=http://localhost:8888/redirect


<img src="./screen_authorization_pocket.png" width="450"/>

Cliquer sur Autoriser.

### Etape 3: Récupérer le token d'accès

In [71]:
USER_CREDENTIALS = Pocket.get_credentials(consumer_key=CONSUMER_KEY, code=REQUEST_TOKEN)
print(USER_CREDENTIALS)

{'access_token': 'bffc0c64-f7f8-fcbd-56ea-1c87d8', 'username': 'toutnouveau@gmail.com'}


In [72]:
ACCESS_TOKEN = USER_CREDENTIALS['access_token']
print(ACCESS_TOKEN)

bffc0c64-f7f8-fcbd-56ea-1c87d8


## Chargement de données sur le nouveau compte

Comme vous venez de créer un compte, vous n'avez pas encore d'articles sauvegardés. On vous a préparé un peu moins de 500 articles (format json). Un tiers de ces articles sont taggés (catégorisés). Un article peut comprendre un ou plusieurs tags.

On vous rappelle que l'objectif à termes sera de prédire les meilleurs tags pour les articles non taggés, étant donnés les mots qui caractérisent ces articles (titre, résumé, et ensemble des mots présents dans le html de la page).

### Chargement du fichier json en mémoire

In [73]:
import json
from pprint import pprint

with open('./data_pocket.json') as fp:    
    data = json.load(fp)

pprint(data)

{'1003565100': {'authors': {'55834601': {'author_id': '55834601',
                                         'item_id': '1003565100',
                                         'name': 'Grafikart.fr',
                                         'url': ''}},
                'excerpt': 'Ionic est un framework qui va vous permettre de '
                           'créer des applications mobiles en utilisant des '
                           'technologies Web. Ionic se base pour cela sur '
                           "d'autres frameworks / technologies qui ont fait "
                           'leurs preuves.  Avant de pouvoir commencer, il '
                           'nous faut évidemment commencer par installer '
                           "l'outil.",
                'favorite': '0',
                'given_title': 'Tutoriel Vidéo Apache Cordova Ionic Framework',
                'given_url': 'https://www.grafikart.fr/tutoriels/cordova/ionic-framework-641',
                'has_image': '0',
      

 '1197225394': {'excerpt': "Beaucoup ont envie d'entreprendre, mais manquent "
                           "d'idées viables ou ont du mal à trouver de bons "
                           'associés.  On réunit pour vous des dizaines de '
                           'personnes complémentaires et qualifiées.',
                'favorite': '0',
                'given_title': '',
                'given_url': 'https://locomotiv.co/',
                'has_image': '1',
                'has_video': '0',
                'image': {'height': '0',
                          'item_id': '1197225394',
                          'src': 'https://locomotiv.co/user/pages/01.home/_afterprogram/shema%20site%202.png',
                          'width': '0'},
                'images': {'1': {'caption': '',
                                 'credit': '',
                                 'height': '0',
                                 'image_id': '1',
                                 'item_id': '1197225394',
          

                'word_count': '1136'},
 '1358791105': {'excerpt': '',
                'favorite': '0',
                'given_title': 'Coworking - NUMA Paris',
                'given_url': 'https://paris.numa.co/coworking-et-communautes/',
                'has_image': '0',
                'has_video': '0',
                'is_article': '0',
                'is_index': '0',
                'item_id': '1358791105',
                'resolved_id': '1358791105',
                'resolved_title': 'Coworking',
                'resolved_url': 'https://paris.numa.co/coworking-et-communautes/',
                'sort_id': 259,
                'status': '0',
                'tags': {'numa': {'item_id': '1358791105', 'tag': 'numa'}},
                'time_added': '1498132272',
                'time_favorited': '0',
                'time_read': '0',
                'time_updated': '1498132278',
                'word_count': '0'},
 '135909267': {'authors': {'1813252': {'author_id': '1813252',
       

                'is_article': '0',
                'is_index': '1',
                'item_id': '1467659795',
                'resolved_id': '596053261',
                'resolved_title': 'VizEat - Book immersive food events: '
                                  'cooking classes, food tours, dinners, '
                                  'brunch, private parties... - VizEat',
                'resolved_url': 'https://www.vizeat.com/',
                'sort_id': 31,
                'status': '0',
                'time_added': '1507006776',
                'time_favorited': '0',
                'time_read': '0',
                'time_updated': '1507006793',
                'videos': {'1': {'height': '0',
                                 'item_id': '1467659795',
                                 'src': 'https://www.vizeat.com/77a80f3f8f5b74fc4d60dbdfe3d785c6.mp4',
                                 'type': '5',
                                 'vid': '',
                                 'video_id

                'given_url': 'https://tympanus.net/Development/SearchUIEffects/index10.html',
                'has_image': '0',
                'has_video': '0',
                'is_article': '0',
                'is_index': '0',
                'item_id': '1599726800',
                'resolved_id': '1599726800',
                'resolved_title': 'Inspiration for Search UI Effects | Demo 10 '
                                  '| Codrops',
                'resolved_url': 'https://tympanus.net/Development/SearchUIEffects/index10.html',
                'sort_id': 361,
                'status': '0',
                'tags': {'search': {'item_id': '1599726800', 'tag': 'search'},
                         'web design': {'item_id': '1599726800',
                                        'tag': 'web design'}},
                'time_added': '1486976983',
                'time_favorited': '0',
                'time_read': '0',
                'time_updated': '1486976991',
                'word_coun

...

                                 'image_id': '6',
                                 'item_id': '1880551559',
                                 'src': 'https://cdn-images-1.medium.com/max/1600/1*-8d_i7UdJ2WvsDg-lGurEA.png',
                                 'width': '0'}},
                'is_article': '1',
                'is_index': '0',
                'item_id': '1880551559',
                'resolved_id': '1880551559',
                'resolved_title': 'Learning to Code, Open Source & Doctor\xa0'
                                  'Who',
                'resolved_url': 'https://codeburst.io/open-source-doctor-who-a666e2ab2d06',
                'sort_id': 135,
                'status': '0',
                'time_added': '1504621622',
                'time_favorited': '0',
                'time_read': '0',
                'time_updated': '1504621622',
                'word_count': '650'},
 '1880557355': {'authors': {'73241239': {'author_id': '73241239',
                                  

                'sort_id': 21,
                'status': '0',
                'tags': {'lewagon': {'item_id': '1913714059', 'tag': 'lewagon'},
                         'react': {'item_id': '1913714059', 'tag': 'react'},
                         'setup': {'item_id': '1913714059', 'tag': 'setup'}},
                'time_added': '1507149466',
                'time_favorited': '0',
                'time_read': '0',
                'time_updated': '1507149478',
                'word_count': '0'},
 '1913816984': {'excerpt': "Stay ahead with the world's most comprehensive "
                           'technology and business learning platform. With '
                           'Safari, you learn the way you learn best. Get '
                           'unlimited access to videos, live online training, '
                           'learning paths, books, tutorials, and more. Start '
                           'Free Trial No credit card required Chapter 4.',
                'favorite': '0',
   

                              'src': 'https://cdn.technologyreview.com/i/images/04.jpg',
                              'width': '0'},
                       '14': {'caption': '',
                              'credit': '',
                              'height': '0',
                              'image_id': '14',
                              'item_id': '299487',
                              'src': 'https://cdn.technologyreview.com/i/images/8swsoftware-algorithm-check1.jpg?sw=280&cx=0&cy=139&cw=3232&ch=1818',
                              'width': '0'},
                       '15': {'caption': '',
                              'credit': '',
                              'height': '0',
                              'image_id': '15',
                              'item_id': '299487',
                              'src': 'https://s3.amazonaws.com/files.technologyreview.com/p/pub/images/samsung-logo.png',
                              'width': '0'},
                       '16': {'caption

               'word_count': '742'},
 '560509859': {'excerpt': '',
               'favorite': '0',
               'given_title': 'color font - Google Search',
               'given_url': 'https://www.google.fr/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8#q=rails+devise+example+host+production.rb&*',
               'has_image': '0',
               'has_video': '0',
               'is_article': '0',
               'is_index': '0',
               'item_id': '560509859',
               'resolved_id': '560509859',
               'resolved_title': 'Google',
               'resolved_url': 'https://www.google.fr/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8#q=consultant+SEO+fabrique+du+net',
               'sort_id': 313,
               'status': '0',
               'time_added': '1490952232',
               'time_favorited': '0',
               'time_read': '0',
               'time_updated': '1490952232',
               'word_count': '0'},
 '582021449': {'excerpt': 'Get the late

                               'credit': '',
                               'height': '0',
                               'image_id': '5',
                               'item_id': '80366476',
                               'src': 'https://static.apiary.io/assets/2qpno0og.svg',
                               'width': '0'},
                         '6': {'caption': '',
                               'credit': '',
                               'height': '0',
                               'image_id': '6',
                               'item_id': '80366476',
                               'src': 'https://static.apiary.io/assets/3CrEtHSd.svg',
                               'width': '0'},
                         '7': {'caption': '',
                               'credit': '',
                               'height': '0',
                               'image_id': '7',
                               'item_id': '80366476',
                               'src': 'https://static.apiary.io/a

### Exercice 1

Chargement des données du json dans le compte nouvellement créé.

Pour communiquer avec une API, il faut envoyer des requêtes HTTP. Ici on veut ajouter les données du fichier json ("given_url" et "tags") dans le compte Pocket.

Que nous dit la doc ? Consulter https://getpocket.com/developer/docs/v3/add.

Par exemple, pour l'ajout d'un seul item, la doc nous donne l'url à laquelle il faut envoyer une requête (https://getpocket.com/v3/add), et la méthode qu'il faut employer. Ici, il s'agit d'une méthode [POST](https://en.wikipedia.org/wiki/POST_(HTTP)). Pour l'ajout de plusieurs items, il faut consulter https://getpocket.com/developer/docs/v3/modify. Il faut aussi une méthode POST. 

Pour envoyer une requête en python, il y a plusieurs solutions. Un des plus simples consiste à utiliser la librairie [requests](http://docs.python-requests.org/en/master/user/quickstart/#make-a-request).

A vous de jouer ! Commencez par un seul (par exemple http://docs.python-requests.org/en/master/user/quickstart/ avec les tags "python, requests"), puis si cela a fonctionné, vous pouvez uploader tout le json. Indice pour passer plusieurs items : penser à encoder les données de la requete en JSON, comme indiqué dans la doc. La documentation de requests indique comment faire ici : http://docs.python-requests.org/en/master/user/quickstart/#more-complicated-post-requests, json = ...

Pour voir si cela a marché, il suffit d'aller sur votre compte Pocket :)

### Exercice 1 - correction 

#### Ajouter un seul item

In [74]:
import requests

data_test = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"url":"http://docs.python-requests.org/en/master/user/quickstart/",
"tags": "python, requests"}

one_item = requests.post('https://getpocket.com/v3/add', data = data_test)

#### Ajouter l'ensemble des items

In [75]:
list_action_add = []
for k,v in data.items():
    if  'tags' in v:
        list_action_add.append({'action':'add', 
                                'url': v['given_url'], 
                                'tags': list(v['tags'].keys())
                               })
list_action_add[0:10]

[{'action': 'add',
  'tags': ['python'],
  'url': 'http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html'},
 {'action': 'add',
  'tags': ['boilerplate', 'lewagon', 'react', 'redux', 'router'],
  'url': 'https://github.com/lewagon/redux-router-boilerplate'},
 {'action': 'add',
  'tags': ['boilerplate', 'lewagon', 'react', 'redux'],
  'url': 'https://github.com/lewagon/redux-boilerplate'},
 {'action': 'add',
  'tags': ['boilerplate', 'lewagon', 'react'],
  'url': 'https://github.com/lewagon/react-boilerplate'},
 {'action': 'add',
  'tags': ['api', 'isomorphic'],
  'url': 'https://www.safaribooksonline.com/library/view/building-isomorphic-javascript/9781491932926/ch04.html'},
 {'action': 'add',
  'tags': ['lewagon', 'react', 'setup'],
  'url': 'https://github.com/lewagon/react-redux-challenges/tree/master/01-Tooling/01-Setup'},
 {'action': 'add',
  'tags': ['mailoop'],
  'url': 'https://console.cloud.google.com/apis/api/gmail.googleapis.com/overview?project=parking-spot-1612

In [76]:
payload = {'consumer_key':CONSUMER_KEY, 
           'access_token':ACCESS_TOKEN,
           'actions': list_action_add}

In [77]:
r = requests.post('https://getpocket.com/v3/send', json = payload)
r

<Response [200]>

Vous pouvez aller voir votre compte, presque 500 items à analyser :)

## Récupération des données disponibles dans l'API

### Exercice 2

Récupérer les urls et les tags des items qui contiennent le tag "python".

C'est par ici : https://getpocket.com/developer/docs/v3/retrieve. A vous de jouer !

### Exercice 2 - correction

In [153]:
import requests
items = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"tag": "python",
"detailType":"complete"}

exo2_items = requests.post('https://getpocket.com/v3/get', data = items)
exo2_list = [v for k,v in exo2_items.json()['list'].items()]

In [154]:
[v['given_url'] for k,v in exo2_items.json()['list'].items()]

['http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html',
 'https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/',
 'https://qbox.io/blog/building-an-elasticsearch-index-with-python',
 'http://blog.fouadhamdi.com/introduction-a-nltk/',
 'http://www.nltk.org/book/ch03.html',
 'https://marcobonzanini.com/2015/06/16/mining-twitter-data-with-python-and-js-part-7-geolocation-and-interactive-maps/',
 'http://www.mikesboyle.com/post/117202964694/python-nltk-wtf-chapter-1-notes-on-things-that',
 'https://pythonprogramming.net/',
 'http://nbviewer.jupyter.org/github/ptwobrussell/Mining-the-Social-Web-2nd-Edition/tree/master/ipynb/',
 'https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/',
 'https://blog.rstudio.org/2016/03/29/feather/']

In [155]:
[', '.join(list(v['tags'].keys())) for k,v in exo2_items.json()['list'].items()]

['python',
 'elastic-search, elasticsearch, python, tutorial',
 'elastic-search, python',
 'french, nlp, nltk, python, tokenize, tokenizer',
 'nltk, python, tokenize',
 'python, tagerstreet',
 'nlp, python',
 'python',
 'python',
 'python',
 'python']

### Exercice 3

Récupérer les urls et les titres des items qui contiennent le mot "python" dans le titre ou l'url

### Exercice 3 - correction

In [156]:
items = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"search": "python",
"detailType":"complete"}

exo3_items = requests.post('https://getpocket.com/v3/get', data = items)
exo3_list = [v for k,v in exo3_items.json()['list'].items()]

In [157]:
[v['given_url'] for k,v in exo3_items.json()['list'].items()]

['http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html',
 'https://www.digitalocean.com/community/tutorials/how-to-install-the-anaconda-python-distribution-on-ubuntu-16-04',
 'http://okomestudio.net/biboroku/?p=2375',
 'https://blog.dominodatalab.com/ab-testing-with-hierarchical-models-in-python/',
 'https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/',
 'https://qbox.io/blog/building-an-elasticsearch-index-with-python',
 'https://marcobonzanini.com/2015/06/16/mining-twitter-data-with-python-and-js-part-7-geolocation-and-interactive-maps/',
 'http://www.mikesboyle.com/post/117202964694/python-nltk-wtf-chapter-1-notes-on-things-that',
 'https://pythonprogramming.net/',
 'https://jakevdp.github.io/blog/2015/08/14/out-of-core-dataframes-in-python/',
 'https://pypi.python.org/pypi/munkres/',
 'https://people.duke.edu/~ccc14/sta-663/Optimization_Bakeoff.html',
 'http://stackoverflow.com/questions/9663918/how-can-i-tag-and-chunk-french-text-using-nltk-and-py

In [158]:
[v['resolved_title'] for k,v in exo3_items.json()['list'].items()]

['Types et variables du langage python¶',
 'How To Install the Anaconda Python Distribution on Ubuntu 16.04',
 'Interpreting A/B Test using Python',
 'A/B Testing with Hierarchical Models in Python',
 'Python + Elasticsearch. First steps.',
 'Build an Elasticsearch Index with Python—Machine Learning Series, Part 1',
 'Mining Twitter Data with Python (and JS) – Part 7: Geolocation and Interactive Maps',
 'Python NLTK WTF, Chapter 1: Notes on things that don’t work right',
 'Python Programming Tutorials',
 'Out-of-Core Dataframes in Python: Dask and OpenStreetMap',
 'munkres 1.0.9',
 'Optimization bake-off¶',
 'How can I tag and chunk French text using NLTK and Python?',
 'Feather: A Fast On-Disk Format for Data Frames for R and Python, powered by Apache Arrow',
 'Setting Up Sublime Text 3 for Full Stack Python Development']

Dans le fichier qui servira à la catégorisation automatique, nous allons avoir besoin : de l'url, du titre, de l'extrait, des tags et du contenu.
    
L'api ne permet pas d'accéder au contenu des sites épinglés. Mais nous pouvons le récupérer grâce à l'url.

### Exercice 4

Constituer d'abord un DataFrame avec les champs resolved_url, resolved_title, excerpt, et les tags des items comprenant le terme python.

### Exercice 4 - correction

In [159]:
items = {"consumer_key":CONSUMER_KEY,
"access_token":ACCESS_TOKEN,
"detailType":"complete"}

exo4_items = requests.post('https://getpocket.com/v3/get', data = items)
len(exo4_items.json()['list'])

469

In [160]:
exo4_list = [v for k,v in exo4_items.json()['list'].items() if 'Python' in v['excerpt']]

In [161]:
import pandas as p

df2 = p.DataFrame(exo2_list)[['item_id','resolved_url','resolved_title','excerpt','tags']]
df3 = p.DataFrame(exo3_list)[['item_id','resolved_url','resolved_title','excerpt','tags']]
df4 = p.DataFrame(exo4_list)[['item_id','resolved_url','resolved_title','excerpt','tags']]

In [162]:
import numpy as np
df = p.merge(df2, df3, on='item_id', how='outer', suffixes=('', '_df3'))
df = p.merge(df,  df4, on='item_id', how='outer', suffixes=('', '_df4'))

def complete_data(x,y,z):
    if x != x:
        if y != y:
            return z
        else:
            return y
    else:
        return x

df['url'] = np.vectorize(complete_data)(df['resolved_url'], df['resolved_url_df3'], df['resolved_url_df4'])
df['title'] = np.vectorize(complete_data)(df['resolved_title'], df['resolved_title_df3'], df['resolved_title_df4'])
df['excerpt'] = np.vectorize(complete_data)(df['excerpt'], df['excerpt_df3'], df['excerpt_df4'])
df['tags'] = np.vectorize(complete_data)(df['tags'], df['tags_df3'], df['tags_df4'])
df = df[['item_id','url','title','excerpt', 'tags']]
df['tags'] = df['tags'].apply(lambda x: ','.join(list(x.keys())) if x==x else x)
df

,item_id,url,title,excerpt,tags
0,1883956314,http://www.xavierdupre.fr/app/teachpyx/helpsph...,Types et variables du langage python¶,Il est impossible d’écrire un programme sans u...,python
1,1395948696,https://tryolabs.com/blog/2015/02/17/python-el...,Python + Elasticsearch. First steps.,"Lately, here at Tryolabs, we started gaining i...","elastic-search,elasticsearch,python,tutorial"
2,1064802343,https://qbox.io/blog/building-an-elasticsearch...,Build an Elasticsearch Index with Python—Machi...,"In this first article, we're going to set up s...","elastic-search,python"
3,1057169119,http://blog.fouadhamdi.com/introduction-a-nltk/,Introduction à l'analyse de texte avec nltk - ...,nltk est une librairie python très utile pour ...,"french,nlp,nltk,python,tokenize,tokenizer"
4,241420475,http://www.nltk.org/book/ch03.html,3 Processing Raw Text,The most important source of texts is undoubte...,"nltk,python,tokenize"
5,957402029,https://marcobonzanini.com/2015/06/16/mining-t...,Mining Twitter Data with Python (and JS) – Par...,Geolocation is the process of identifying the ...,"python,tagerstreet"
6,1072497525,http://www.mikesboyle.com/post/117202964694/py...,"Python NLTK WTF, Chapter 1: Notes on things th...","If you’re reading this post, you’re probably a...","nlp,python"
7,680797791,https://pythonprogramming.net/,Python Programming Tutorials,"Learn how to use Python with Pandas, Matplotli...",python
8,1056127688,http://nbviewer.jupyter.org/github/ptwobrussel...,Jupyter Notebook Viewer,"Delivered by Fastly, Rendered by Rackspace nbv...",python
9,378831480,https://jakevdp.github.io/blog/2013/06/15/numb...,Numba vs. Cython: Take 2,Last summer I wrote a post comparing the perfo...,python


## Compléter les données avec du webscraping

### Mais c'est quoi le webscraping ? 

On vous explique tout ici :  
- [Définition](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_Eco_Web_Scraping.html#a-eco-web-scraping)
- [Un détour par le Web : comment fonctionne un site ?](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_Eco_Web_Scraping.html#un-detour-par-le-web-comment-fonctionne-un-site)
- [Scrapper avec python](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_Eco_Web_Scraping.html#scrapper-avec-python)

### Beautiful Soup 

In [21]:
! pip install bs4

  Using cached beautifulsoup4-4.6.0-py3-none-any.whl


Reprenons notre liste de sites sur python et analysons le contenu HTML.

In [22]:
from bs4 import BeautifulSoup

In [163]:
from pprint import pprint

#première url de la liste
url = df['url'].iloc[0]
print(url)

# récupération du contenu (même librairie que pour les requetes api => 
#requests est une librairie qui permet de faire des requetes http, que cela soit pour des api ou du webscraping)

r = requests.get(url)

#pprint : librairie pour "pretty print" (essayer sans : on voit pas grand chose)
pprint(r.text)

http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html
('\n'
 '<!DOCTYPE html>\n'
 '\n'
 '<html xmlns="http://www.w3.org/1999/xhtml" lang="fr">\n'
 '  <head>\n'
 '    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n'
 '    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n'
 '    <title>Types et variables du langage python &#8212; Programmation avec '
 'le langage Python</title>\n'
 '    <link rel="stylesheet" href="../_static/bootstrap-sphinx.css" '
 'type="text/css" />\n'
 '    <link rel="stylesheet" href="../_static/pygments.css" type="text/css" '
 '/>\n'
 '    <link rel="stylesheet" href="../_static/style_notebook_snippet.css" '
 'type="text/css" title="style_figure_notebook" />\n'
 '    <link rel="stylesheet" href="../_static/my-styles.css" type="text/css" '
 '/>\n'
 '    <script type="text/javascript">\n'
 '      var DOCUMENTATION_OPTIONS = {\n'
 "        URL_ROOT:    '../',\n"
 "        VERSION:     '0.1.177',\n"
 '        COLLAPSE_INDE

 '<p><code class="docutils literal"><span class="pre">n</span></code> est le '
 'nombre de chiffres total et <code class="docutils literal"><span '
 'class="pre">d</span></code> est le nombre de dÃ©cimales,\n'
 '<code class="docutils literal"><span class="pre">f</span></code> dÃ©signe un '
 'format rÃ©el indiquÃ© par la prÃ©sence du symbole <code class="docutils '
 'literal"><span class="pre">%</span></code>.</p>\n'
 '<p>Exemple :</p>\n'
 '<p>&gt;&gt;&gt;</p>\n'
 '<div class="highlight-default"><div '
 'class="highlight"><pre><span></span><span class="n">x</span> <span '
 'class="o">=</span> <span class="mf">0.123456789</span>\n'
 '<span class="nb">print</span><span class="p">(</span><span '
 'class="n">x</span><span class="p">)</span>             <span class="c1"># '
 'affiche 0.123456789</span>\n'
 '<span class="nb">print</span><span class="p">(</span><span '
 'class="s2">&quot;</span><span class="si">%1.2f</span><span '
 'class="s2">&quot;</span> <span class="o">%</span> <span '
 'c

In [164]:
# on stock le contenu html dans la variable html
html = r.text

# on "parse" le html grâce à la librairie beautiful soup
soup = BeautifulSoup(html, "html5lib")

# c'est plus "joli" encore que pprint et surtout, 
# il y a plein de méthodes pour extraire les informations que l'on souhaite
soup

<!DOCTYPE html>
<html lang="fr" xmlns="http://www.w3.org/1999/xhtml"><head>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
    <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
    <title>Types et variables du langage python — Programmation avec le langage Python</title>
    <link href="../_static/bootstrap-sphinx.css" rel="stylesheet" type="text/css"/>
    <link href="../_static/pygments.css" rel="stylesheet" type="text/css"/>
    <link href="../_static/style_notebook_snippet.css" rel="stylesheet" title="style_figure_notebook" type="text/css"/>
    <link href="../_static/my-styles.css" rel="stylesheet" type="text/css"/>
    <script type="text/javascript">
      var DOCUMENTATION_OPTIONS = {
        URL_ROOT:    '../',
        VERSION:     '0.1.177',
        COLLAPSE_INDEX: false,
        FILE_SUFFIX: '.html',
        HAS_SOURCE:  true,
        SOURCELINK_SUFFIX: '.txt'
      };
    </script>
    <script src="../_static/jquery.js" type="text/javascript"><

In [165]:
type(soup)

bs4.BeautifulSoup

_soup_ est une instance de la classe BeautifulSoup (que l'on a importé de la librairie bs4). C'est une représentation du code html avec des méthodes pratiques, telles que __find__. __find__ permet de trouver la première occurence d'une balise html qu'on lui passe. Par exemple le 1er lien de la page :

In [166]:
soup.find("a")

<a class="navbar-brand" href="../index.html"><span><img src="../_static/project_ico_small.png"/></span>
          .</a>

Si on veut tous les liens, il faut utiliser __findAll__. Cela renvoie une objet qui se comporte comme une liste. On peut itérer dessus, ou facilement le transformer en objet "list" (en faisant list())

In [167]:
print(type(soup.findAll("a")))
soup.findAll("a")

<class 'bs4.element.ResultSet'>


[<a class="navbar-brand" href="../index.html"><span><img src="../_static/project_ico_small.png"/></span>
           .</a>,
 <a href="http://www.xavierdupre.fr">XD</a>,
 <a href="../blog/main_0000.html">blog</a>,
 <a href="../genindex.html">index</a>,
 <a data-target="#" data-toggle="dropdown" href="../index.html" id="dLabelGlobalToc" role="button">Site <b class="caret"></b></a>,
 <a class="reference internal" href="../introduction.html">Introduction</a>,
 <a class="reference internal" href="../introduction.html">Introduction</a>,
 <a class="reference internal" href="index.html">Langage, variable et fonctions</a>,
 <a class="reference internal" href="../c_classes/index.html">Classes et programmation objet</a>,
 <a class="reference internal" href="../c_exception/index.html">Exceptions</a>,
 <a class="reference internal" href="../c_io/index.html">EntrÃ©es Sorties</a>,
 <a class="reference internal" href="../c_module/index.html">Modules</a>,
 <a class="reference internal" href="../c_regex/

On peut sélectionner des tags qui ont certains attributs css. Par exemple ici, on peut vouloir seulement les liens internes du site. Ils ont la classe "internal". Les autres "external".

In [168]:
list(soup.findAll("a", {"class": "internal"}))

[<a class="reference internal" href="../introduction.html">Introduction</a>,
 <a class="reference internal" href="../introduction.html">Introduction</a>,
 <a class="reference internal" href="index.html">Langage, variable et fonctions</a>,
 <a class="reference internal" href="../c_classes/index.html">Classes et programmation objet</a>,
 <a class="reference internal" href="../c_exception/index.html">Exceptions</a>,
 <a class="reference internal" href="../c_io/index.html">EntrÃ©es Sorties</a>,
 <a class="reference internal" href="../c_module/index.html">Modules</a>,
 <a class="reference internal" href="../c_regex/index.html">Expression rÃ©guliÃ¨res</a>,
 <a class="reference internal" href="../c_parallelisation/index.html">ParallÃ©lisation</a>,
 <a class="reference internal" href="../c_gui/index.html">Interfaces graphiques</a>,
 <a class="reference internal" href="index.html">Langage, variable et fonctions</a>,
 <a class="reference internal" href="../c_classes/index.html">Classes et progra

Ce n'est pas toujours comme ça, la plupart du temps, il faudra la forme de la valeur de l'attribut href. les liens externes pourront etre identifiés parce qu'ils sont absolus (on indique l'ensemble du lien comme href="https://docs.python.org/3/reference/expressions.html etc. et non pas un lien relatif comme href="../c_exception/exception.html ('..' signifie "le répertoire parent).

On peut lister toutes les balises h1 et h2 de cette page, en une ligne.

In [169]:
list(soup.findAll({"h1", "h2"}))

[<h1>Types et variables du langage python<a class="headerlink" href="#types-et-variables-du-langage-python" title="Lien permanent vers ce titre">Â¶</a></h1>,
 <h2><a class="toc-backref" href="#id21">Variables</a><a class="headerlink" href="#variables" title="Lien permanent vers ce titre">Â¶</a></h2>,
 <h2><a class="toc-backref" href="#id22">Types immuables (ou immutable)</a><a class="headerlink" href="#types-immuables-ou-immutable" title="Lien permanent vers ce titre">Â¶</a></h2>,
 <h2><a class="toc-backref" href="#id24">Nombres rÃ©els et entiers</a><a class="headerlink" href="#nombres-reels-et-entiers" title="Lien permanent vers ce titre">Â¶</a></h2>,
 <h2><a class="toc-backref" href="#id26">ChaÃ®ne de caractÃ¨res</a><a class="headerlink" href="#chaine-de-caracteres" title="Lien permanent vers ce titre">Â¶</a></h2>,
 <h2><a class="toc-backref" href="#id33">Types modifiables ou mutable</a><a class="headerlink" href="#types-modifiables-ou-mutable" title="Lien permanent vers ce titre">Â¶

Naviguer vers les enfants

In [170]:
child = soup.find("h2").a
child

<a class="toc-backref" href="#id21">Variables</a>

Retrouver le parent

In [171]:
child.parent

<h2><a class="toc-backref" href="#id21">Variables</a><a class="headerlink" href="#variables" title="Lien permanent vers ce titre">Â¶</a></h2>

Récupérer tous les enfants

In [172]:
children = soup.find("h2").findAll("a")
children

[<a class="toc-backref" href="#id21">Variables</a>,
 <a class="headerlink" href="#variables" title="Lien permanent vers ce titre">Â¶</a>]

Récupérer les attributs des éléments html

In [173]:
[c.attrs for c in children]

[{'class': ['toc-backref'], 'href': '#id21'},
 {'class': ['headerlink'],
  'href': '#variables',
  'title': 'Lien permanent vers ce titre'}]

Accéder à la valeur d'un attribut en particulier

In [174]:
[c.attrs['href'] for c in children]

['#id21', '#variables']

Jusqu'ici, on passe systématiquement par une balise html. Comment faire si on veut récupérer les élements qui ont une class css, quelle que soit la balise html ?

In [175]:
internals = soup.findAll("", {"class": "internal"})
internals

[<a class="reference internal" href="../introduction.html">Introduction</a>,
 <a class="reference internal" href="../introduction.html">Introduction</a>,
 <a class="reference internal" href="index.html">Langage, variable et fonctions</a>,
 <a class="reference internal" href="../c_classes/index.html">Classes et programmation objet</a>,
 <a class="reference internal" href="../c_exception/index.html">Exceptions</a>,
 <a class="reference internal" href="../c_io/index.html">EntrÃ©es Sorties</a>,
 <a class="reference internal" href="../c_module/index.html">Modules</a>,
 <a class="reference internal" href="../c_regex/index.html">Expression rÃ©guliÃ¨res</a>,
 <a class="reference internal" href="../c_parallelisation/index.html">ParallÃ©lisation</a>,
 <a class="reference internal" href="../c_gui/index.html">Interfaces graphiques</a>,
 <a class="reference internal" href="index.html">Langage, variable et fonctions</a>,
 <a class="reference internal" href="../c_classes/index.html">Classes et progra

On a presque fini de passer en revue la librairie. Il reste la notion de "sibling", pratique pour parcourir un tableau. nextSibling : permet de récupérer l'élément html suivant et "de même niveau" dans l'arbre (le prochain frère), previousSibling, le précédent. findChildren permet de trouver tous les enfants.

Enfin la méthode "get_text()" pour récupérer le contenu des balises html. A appliquer en dernier ! En effet, une fois appliquer, on perd toute trace à l'arbre html. SI vous voulez aller plus loin, la doc est bien faite : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

### Exercice 5

Récupérer le contenu des code snippets de la page : http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html

### Exercice 5 - correction

In [176]:
code_snippets = soup.findAll('pre')
code_snippets[0].get_text()

'n = 11\nsomme = 0                   # initialisation : la somme est nulle\nfor i in range(1, n):       # pour tous les indices de 1 Ã\xa0 n exclu\n    somme = somme + i       # on ajoute le i Ã¨me Ã©lÃ©ment Ã\xa0 somme\nprint(somme)\n'

### Exercice 6

Récupérer la 3ème colonne (intitulée "exemples") du tableau des opérateurs : http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html

Astuce : explorer les CSS selectors : https://www.crummy.com/software/BeautifulSoup/bs4/doc/.

### Exercice 6 - correction

In [224]:
table = soup.find('tbody')
rows = table.findAll('tr')
[r.select_one("td:nth-of-type(3)").get_text() for r in rows]

['x = 8 << 1',
 'x = 8 | 1',
 'x = 11 & 2',
 'x = y + z',
 'x += 3',
 'x = y * z',
 'x = y // 3',
 'x = y % 3',
 'x *= 3',
 'x = y ** 3']

### Exercice 7

Récupérer le contenu des titres (h1, h2, h3, etc.) et des balise p de l'ensemble des liens de notre liste et compter le nombre d'occurences du mot python, et la porportion que cela représente dans l'ensemble des mots

### Exercice 7 - correction 

In [177]:
def nb_words(url):
    print(url)
    try: 
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html5lib")
        nb_python = 0
        nb_words = 0
        for e in soup.findAll({'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p'}):
            text = e.get_text()
            words = text.split(' ')
            nb_words += len(words)
            python_words = [w for w in words if "python" in w.lower()]
            nb_python += len(python_words)
        return (nb_python, nb_words)
    except:
        return "scraper banned"

In [178]:
df['python_occ'] = df['url'].apply(lambda x: nb_words(x))

http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html
https://tryolabs.com/blog/2015/02/17/python-elasticsearch-first-steps/
https://qbox.io/blog/building-an-elasticsearch-index-with-python
http://blog.fouadhamdi.com/introduction-a-nltk/
http://www.nltk.org/book/ch03.html
https://marcobonzanini.com/2015/06/16/mining-twitter-data-with-python-and-js-part-7-geolocation-and-interactive-maps/
http://www.mikesboyle.com/post/117202964694/python-nltk-wtf-chapter-1-notes-on-things-that
https://pythonprogramming.net/
http://nbviewer.jupyter.org/github/ptwobrussell/Mining-the-Social-Web-2nd-Edition/tree/master/ipynb/
https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/
https://blog.rstudio.org/2016/03/29/feather/
https://www.digitalocean.com/community/tutorials/how-to-install-the-anaconda-python-distribution-on-ubuntu-16-04
http://okomestudio.net/biboroku/?p=2375
https://blog.dominodatalab.com/ab-testing-with-hierarchical-models-in-python/
https://jakevdp.github.io/blog

In [179]:
df

,item_id,url,title,excerpt,tags,python_occ
0,1883956314,http://www.xavierdupre.fr/app/teachpyx/helpsph...,Types et variables du langage python¶,Il est impossible d’écrire un programme sans u...,python,"(29, 5164)"
1,1395948696,https://tryolabs.com/blog/2015/02/17/python-el...,Python + Elasticsearch. First steps.,"Lately, here at Tryolabs, we started gaining i...","elastic-search,elasticsearch,python,tutorial","(11, 1509)"
2,1064802343,https://qbox.io/blog/building-an-elasticsearch...,Build an Elasticsearch Index with Python—Machi...,"In this first article, we're going to set up s...","elastic-search,python","(11, 1901)"
3,1057169119,http://blog.fouadhamdi.com/introduction-a-nltk/,Introduction à l'analyse de texte avec nltk - ...,nltk est une librairie python très utile pour ...,"french,nlp,nltk,python,tokenize,tokenizer",scraper banned
4,241420475,http://www.nltk.org/book/ch03.html,3 Processing Raw Text,The most important source of texts is undoubte...,"nltk,python,tokenize","(58, 11800)"
5,957402029,https://marcobonzanini.com/2015/06/16/mining-t...,Mining Twitter Data with Python (and JS) – Par...,Geolocation is the process of identifying the ...,"python,tagerstreet","(10, 2435)"
6,1072497525,http://www.mikesboyle.com/post/117202964694/py...,"Python NLTK WTF, Chapter 1: Notes on things th...","If you’re reading this post, you’re probably a...","nlp,python","(21, 1475)"
7,680797791,https://pythonprogramming.net/,Python Programming Tutorials,"Learn how to use Python with Pandas, Matplotli...",python,"(6, 93)"
8,1056127688,http://nbviewer.jupyter.org/github/ptwobrussel...,Jupyter Notebook Viewer,"Delivered by Fastly, Rendered by Rackspace nbv...",python,"(0, 195)"
9,378831480,https://jakevdp.github.io/blog/2013/06/15/numb...,Numba vs. Cython: Take 2,Last summer I wrote a post comparing the perfo...,python,"(21, 1183)"
